# Integrating with OPERA's ISCE3 s1-reader

This notebook requires `isce3`, and `s1-reader` to be installed as well as the rest of the packages listed in `requirements.txt`. Install using these commands:
```bash
conda install -c conda-forge isce3
conda install -c conda-forge backoff

git clone https://github.com/opera-adt/s1-reader.git
python -m pip install -e s1-reader
```

In [1]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import s1reader
from s1bursts import utils, metadata, opera

In [2]:
zip_path = Path('../example_data/S1B_IW_SLC__1SDV_20210131T151555_20210131T151621_025400_03067B_415D.zip')
url_path = 'https://datapool.asf.alaska.edu/SLC/SB/S1B_IW_SLC__1SDV_20210131T151555_20210131T151621_025400_03067B_415D.zip'
safe_name = zip_path.with_suffix('.SAFE').name
swath_num = 1
pol = "VV"

orbit_dir = Path('../example_data/sentinel1_orbits')
if not orbit_dir.is_dir():
    orbit_dir.mkdir()

In [3]:
s1reader.s1_orbit.download_orbit(safe_name,orbit_dir)

## Opera Workflow

In [4]:
orbit_path = s1reader.get_orbit_file_from_dir(zip_path, orbit_dir)

In [5]:
# returns the list of the bursts
opera_burst_items = s1reader.load_bursts(zip_path, orbit_path, swath_num, pol)
opera_burst_ids = [x.burst_id for x in opera_burst_items]
opera_burst = opera_burst_items[0]

In [6]:
opera_burst

Sentinel1BurstSlc(sensing_start=datetime.datetime(2021, 1, 31, 15, 15, 56, 832500), radar_center_frequency=5405000454.33435, wavelength=0.05546576, azimuth_steer_rate=0.027757171601738514, azimuth_time_interval=0.002055556299999998, slant_range_time=0.005354305379666438, starting_range=802590.1853264123, iw2_mid_range=878040.0430978123, range_sampling_rate=64345238.12571428, range_pixel_spacing=2.329562114715323, shape=(1502, 21209), azimuth_fm_rate=Poly1d(order= 2, mean=802590, std=1.49896e+08), doppler=Doppler(poly1d=Poly1d(order= 2, mean=803664, std=1.49896e+08), lut2d=<isce3.ext.isce3.core.LUT2d object at 0x18d8cea70>), range_bandwidth=56500000.0, polarization='VV', burst_id='t174_372322_iw1', platform_id='S1B', center=<shapely.geometry.point.Point object at 0x18a079bd0>, border=[<shapely.geometry.polygon.Polygon object at 0x18a079390>], orbit=<isce3.ext.isce3.core.Orbit object at 0x18d8d5bf0>, orbit_direction='Descending', tiff_path='/vsizip/../example_data/S1B_IW_SLC__1SDV_202101

In [7]:
opera_dict = opera_burst.__dict__

## Now with ASF package

In [8]:
auth = utils.get_netrc_auth()
manifest, annotations = metadata.edl_download_metadata(url_path, auth)

In [9]:
slc = metadata.SLCMetadata(url_path, manifest, annotations)
swath = metadata.SwathMetadata(slc, pol, swath_index=swath_num - 1)
asf_burst_list = [metadata.BurstMetadata(swath, x) for x in range(swath.n_bursts)]
asf_burst = asf_burst_list[0]
item = asf_burst.to_stac_item()

In [10]:
asf_burst = opera.stac_item_to_opera_burst(item,'VV',orbit_dir)

In [12]:
asf_dict = asf_burst.__dict__

In [13]:
[x for x in asf_dict if not opera_dict[x]]

['i_burst']

In [14]:
[x for x in asf_dict if type(asf_dict[x]) != type(opera_dict[x])]

[]

In [15]:
# attributes in oks don't work correctly when comparing with ==
oks = ['azimuth_fm_rate','doppler','orbit', 'center', 'border','tiff_path']

[x for x in asf_dict if (asf_dict[x] != opera_dict[x]) & (x not in oks)]

[]

Tada! Check out the `example.ipynb` to see how we can download a burst's data directly using the information contained in a burst STAC item.

## Download data with RemoteSentinel1BurstSLC class

In [21]:
asf_burst = opera.stac_item_to_opera_burst(item,'VV',orbit_dir, remote=True)

In [22]:
asf_burst.tiff_path, asf_burst.url_path

('',
 'https://datapool.asf.alaska.edu/SLC/SB/S1B_IW_SLC__1SDV_20210131T151555_20210131T151621_025400_03067B_415D.zip')

In [23]:
# download into memory
array = asf_burst.edl_download_data()
array.shape

(1502, 21209)

In [24]:
# download to file
asf_burst.slc_to_file('./test.bin', fmt='ENVI')

In [25]:
asf_burst.tiff_path

'./test.bin'

In [26]:
!gdalinfo test.bin

Driver: ENVI/ENVI .hdr Labelled
Files: test.bin
       test.hdr
Size is 21209, 1502
Image Structure Metadata:
  INTERLEAVE=BAND
Corner Coordinates:
Upper Left  (    0.0,    0.0)
Lower Left  (    0.0, 1502.0)
Upper Right (21209.0,    0.0)
Lower Right (21209.0, 1502.0)
Center      (10604.5,  751.0)
Band 1 Block=21209x1 Type=CFloat32, ColorInterp=Undefined
